In [1]:
import numpy as np
from PIL import Image, ImageDraw
from homography_util import *

# Method 3 - 1-step distortion
img1 = np.array(Image.open("1.jpg"))
draw_img1 = Image.open("1.jpg")
draw = ImageDraw.Draw(draw_img1)
lhs = np.zeros((5,5))
rhs = np.zeros((5,1))

# 1st pair of parallel lines
R1 = np.array([1142,817,1])
R2 = np.array([1127,993,1])
S1 = np.array([1142,817,1])
S2 = np.array([1258,763,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 10)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 10)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[0,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[0][0] = - r[2]*s[2]

# 2nd pair of parallel lines
S1 = np.array([1524,637,1])
S2 = np.array([1387,702,1])
R1 = np.array([1524,637,1])
R2 = np.array([1517,835,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 10)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 10)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[1,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[1][0] = - r[2]*s[2]

# 3rd pair of parallel lines
R1 = np.array([1507,1122,1])
R2 = np.array([1511,1001,1])
S1 = np.array([1507,1122,1])
S2 = np.array([1359,1167,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 10)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 10)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[2,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[2][0] = - r[2]*s[2]

# 4th pair of parallel lines
S1 = np.array([1103,1245,1])
S2 = np.array([1112,1140,1])
R1 = np.array([1103,1245,1])
R2 = np.array([1225,1209,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 10)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 10)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[3,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[3][0] = - r[2]*s[2]

# 5th pair of parallel lines
R1 = np.array([1244,946,1])
R2 = np.array([1375,894,1])
S1 = np.array([1244,946,1])
S2 = np.array([1232,1098,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 10)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 10)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[4,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[4][0] = - r[2]*s[2]
draw_img1.save("img1-m3-annotaton.jpg")

c_tmp = np.matmul(np.linalg.inv(lhs), rhs)
c_infinity = np.array([[c_tmp[0][0], c_tmp[1][0], c_tmp[3][0]], 
                       [c_tmp[1][0], c_tmp[2][0], c_tmp[4][0]], 
                       [c_tmp[3][0], c_tmp[4][0], 1]])

sigma, U = np.linalg.eig(c_infinity);
print("c_infinity = ", c_infinity)
print("sigma = ", sigma)
scale = 1e-3
d = np.diag([1/np.sqrt(sigma[0]), 1/np.sqrt(sigma[1]), scale]);
H_D = np.matmul(d, np.transpose(U)); 
H_D[0,:] = H_D[0,:]*(-1)
H_D = H_D/H_D[2,2]
print("inverse(H_D) = ", H_D)

(x_min,y_min,mapped_img1) = Mapped(img1, H_D)
mapped = Image.fromarray(np.uint8(mapped_img1), 'RGB')
mapped.save("img1-m3.jpg")

c_infinity =  [[ 1.69093543e+06 -2.49060899e+06 -1.14168358e+03]
 [-2.49060899e+06  5.08391153e+06  1.84588983e+03]
 [-1.14168358e+03  1.84588983e+03  1.00000000e+00]]
sigma =  [6.40092796e+06 3.73919786e+05 2.10091968e-01]
inverse(H_D) =  [[ 1.84766478e-01 -3.49411964e-01 -1.33718264e-04]
 [ 1.44567270e+00  7.64461340e-01 -6.40217558e-04]
 [ 5.04225109e-04 -1.16064620e-04  1.00000000e+00]]


In [2]:
# Method 3 - Image 2 - 1-step distortion
img1 = np.array(Image.open("2.jpg"))
draw_img1 = Image.open("2.jpg")
draw = ImageDraw.Draw(draw_img1)
lhs = np.zeros((5,5))
rhs = np.zeros((5,1))

# 1st pair of parallel lines
R1 = np.array([237,57,1])
R2 = np.array([284,65,1])
S1 = np.array([237,57,1])
S2 = np.array([237,146,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 5)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 5)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[0,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[0][0] = - r[2]*s[2]

# 2nd pair of parallel lines
S1 = np.array([236,283,1])
S2 = np.array([238,227,1])
R1 = np.array([236,283,1])
R2 = np.array([275,281,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 5)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 5)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[1,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[1][0] = - r[2]*s[2]

# 3rd pair of parallel lines
R1 = np.array([335,277,1])
R2 = np.array([303,278,1])
S1 = np.array([335,277,1])
S2 = np.array([336,230,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 5)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 5)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[2,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[2][0] = - r[2]*s[2]

# 4th pair of parallel lines
S1 = np.array([337,73,1])
S2 = np.array([308,69,1])
R1 = np.array([337,73,1])
R2 = np.array([337,128,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 5)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 5)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[3,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[3][0] = - r[2]*s[2]

# 5th pair of parallel lines
R1 = np.array([399,154,1])
R2 = np.array([398,231,1])
S1 = np.array([399,154,1])
S2 = np.array([443,157,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 5)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 5)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[4,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[4][0] = - r[2]*s[2]
draw_img1.save("img2-m3-annotaton.jpg")

c_tmp = np.matmul(np.linalg.inv(lhs), rhs)
c_infinity = np.array([[c_tmp[0][0], c_tmp[1][0], c_tmp[3][0]], 
                       [c_tmp[1][0], c_tmp[2][0], c_tmp[4][0]], 
                       [c_tmp[3][0], c_tmp[4][0], 1]])

sigma, U = np.linalg.eig(c_infinity);
print("c_infinity = ", c_infinity)
print("sigma = ", sigma)
scale = 1e-3
d = np.diag([1/np.sqrt(sigma[0]), 1/np.sqrt(sigma[1]), scale]);
H_D = np.matmul(d, np.transpose(U)); 
H_D[1,:] = H_D[1,:]*(-1)
H_D = H_D/H_D[2,2]
print("inverse(H_D) = ", H_D)

(x_min,y_min,mapped_img1) = Mapped(img1, H_D)
mapped = Image.fromarray(np.uint8(mapped_img1), 'RGB')
mapped.save("img2-m3.jpg")

c_infinity =  [[5.28057276e+05 1.19822693e+05 5.64334100e+02]
 [1.19822693e+05 1.42237583e+05 1.99379048e+02]
 [5.64334100e+02 1.99379048e+02 1.00000000e+00]]
sigma =  [5.62242068e+05 1.08053438e+05 3.52678424e-01]
inverse(H_D) =  [[ 1.28246981e+00  3.65875305e-01  1.41698883e-03]
 [-8.34595891e-01  2.92542991e+00  1.03911131e-03]
 [-9.28023762e-04 -6.19955418e-04  1.00000000e+00]]


In [3]:
# Method 3 - Image 3 - 1-step distortion
img1 = np.array(Image.open("3.jpg"))
draw_img1 = Image.open("3.jpg")
draw = ImageDraw.Draw(draw_img1)
lhs = np.zeros((5,5))
rhs = np.zeros((5,1))

# 1st pair of parallel lines
R1 = np.array([1407,669,1])
R2 = np.array([1425,1245,1])
S1 = np.array([1407,669,1])
S2 = np.array([2541,525,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[0,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[0][0] = - r[2]*s[2]

# 2nd pair of parallel lines
R1 = np.array([3327,423,1])
R2 = np.array([2478,528,1])
S1 = np.array([3327,423,1])
S2 = np.array([3270,1539,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[1,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[1][0] = - r[2]*s[2]

# 3rd pair of parallel lines
R1 = np.array([3186,2790,1])
R2 = np.array([3240,2067,1])
S1 = np.array([3186,2790,1])
S2 = np.array([2502,2538,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[2,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[2][0] = - r[2]*s[2]

# 4th pair of parallel lines
R1 = np.array([1431,2145,1])
R2 = np.array([1842,2292,1])
S1 = np.array([1431,2145,1])
S2 = np.array([1434,1845,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[3,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[3][0] = - r[2]*s[2]

# 5th pair of parallel lines
R1 = np.array([825,747,1])
R2 = np.array([840,1092,1])
S1 = np.array([825,747,1])
S2 = np.array([1164,705,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[4,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[4][0] = - r[2]*s[2]
draw_img1.save("img3-m3-annotaton.jpg")

c_tmp = np.matmul(np.linalg.inv(lhs), rhs)
c_infinity = np.array([[c_tmp[0][0], c_tmp[1][0], c_tmp[3][0]], 
                       [c_tmp[1][0], c_tmp[2][0], c_tmp[4][0]], 
                       [c_tmp[3][0], c_tmp[4][0], 1]])

sigma, U = np.linalg.eig(c_infinity);
print("c_infinity = ", c_infinity)
print("sigma = ", sigma)
scale = 1e-3
d = np.diag([1/np.sqrt(sigma[0]), 1/np.sqrt(sigma[1]), scale]);
H_D = np.matmul(d, np.transpose(U)); 
H_D[0,:] = H_D[0,:]
H_D[1,:] = H_D[1,:]
H_D = H_D/H_D[2,2]
print("inverse(H_D) = ", H_D)

(x_min,y_min,mapped_img1) = Mapped(img1, H_D)
mapped = Image.fromarray(np.uint8(mapped_img1), 'RGB')
mapped.save("img3-m3.jpg")

c_infinity =  [[ 2.82875111e+06 -1.42113769e+06 -1.47630443e+03]
 [-1.42113769e+06  3.93814279e+06  1.10829901e+03]
 [-1.47630443e+03  1.10829901e+03  1.00000000e+00]]
sigma =  [4.90900315e+06 1.85789157e+06 1.87839813e-01]
inverse(H_D) =  [[-2.54596624e-01  3.72676703e-01  1.60704624e-04]
 [ 6.05785243e-01  4.13846409e-01 -2.34490598e-04]
 [ 4.64766402e-04 -1.13708925e-04  1.00000000e+00]]


In [4]:
# Method 3 - Image 4 - 1-step distortion
img1 = np.array(Image.open("4.jpg"))
draw_img1 = Image.open("4.jpg")
draw = ImageDraw.Draw(draw_img1)
lhs = np.zeros((5,5))
rhs = np.zeros((5,1))

# 1st pair of parallel lines
R1 = np.array([1215,1227,1])
R2 = np.array([1305,1518,1])
S1 = np.array([1215,1227,1])
S2 = np.array([1341,1071,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[0,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[0][0] = - r[2]*s[2]

# 2nd pair of parallel lines
R1 = np.array([1782,567,1])
R2 = np.array([1602,765,1])
S1 = np.array([1782,567,1])
S2 = np.array([1914,909,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[1,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[1][0] = - r[2]*s[2]

# 3rd pair of parallel lines
R1 = np.array([2229,1665,1])
R2 = np.array([2115,1386,1])
S1 = np.array([2229,1665,1])
S2 = np.array([1965,1842,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[2,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[2][0] = - r[2]*s[2]

# 4th pair of parallel lines
R1 = np.array([1500,2166,1])
R2 = np.array([1713,2019,1])
S1 = np.array([1500,2166,1])
S2 = np.array([1410,1875,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[3,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[3][0] = - r[2]*s[2]

# 5th pair of parallel lines
R1 = np.array([495,1269,1])
R2 = np.array([576,1638,1])
S1 = np.array([495,1269,1])
S2 = np.array([624,1047,1])
draw.line((R1[0], R1[1], R2[0], R2[1]), fill=(255,0,0), width = 20)
draw.line((S1[0], S1[1], S2[0], S2[1]), fill=(255,0,0), width = 20)
r = np.cross(R1, R2) ;
s = np.cross(S1, S2) ;
lhs[4,:] = np.array([r[0]*s[0], r[1]*s[0]+r[0]*s[1], r[1]*s[1], r[2]*s[0]+r[0]*s[2], r[2]*s[1]+r[1]*s[2]]) 
rhs[4][0] = - r[2]*s[2]
draw_img1.save("img4-m3-annotaton.jpg")

c_tmp = np.matmul(np.linalg.inv(lhs), rhs)
c_infinity = np.array([[c_tmp[0][0], c_tmp[1][0], c_tmp[3][0]], 
                       [c_tmp[1][0], c_tmp[2][0], c_tmp[4][0]], 
                       [c_tmp[3][0], c_tmp[4][0], 1]])

sigma, U = np.linalg.eig(c_infinity);
print("c_infinity = ", c_infinity)
print("sigma = ", sigma)
scale = 1e-3
d = np.diag([1/np.sqrt(sigma[0]), 1/np.sqrt(sigma[1]), scale]);
H_D = np.matmul(d, np.transpose(U)); 
H_D[0,:] = H_D[0,:]*(-1)
H_D[1,:] = H_D[1,:]
H_D = H_D/H_D[2,2]
print("inverse(H_D) = ", H_D)

(x_min,y_min,mapped_img1) = Mapped(img1, H_D)
mapped = Image.fromarray(np.uint8(mapped_img1), 'RGB')
mapped.save("img4-m3.jpg")

c_infinity =  [[ 1.20957565e+06  1.90062586e+06 -7.90885709e+02]
 [ 1.90062586e+06  2.56344887e+07 -2.73485439e+02]
 [-7.90885709e+02 -2.73485439e+02  1.00000000e+00]]
sigma =  [2.57815012e+07 1.06256376e+06 4.41396146e-01]
inverse(H_D) =  [[ 1.51882644e-02  1.96358951e-01 -2.54886236e-06]
 [-9.67224281e-01  7.48143124e-02  7.00667538e-04]
 [ 7.21100359e-04 -4.27961166e-05  1.00000000e+00]]
